In [1]:
import numpy as np
import pandas as pd

In [ ]:
!pip uninstall numpy 
!pip install numpy

Found existing installation: numpy 1.21.6
Uninstalling numpy-1.21.6:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.8
    /usr/local/lib/python3.8/dist-packages/numpy-1.21.6.dist-info/*
    /usr/local/lib/python3.8/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.8/dist-packages/numpy.libs/libopenblasp-r0-2d23e62b.3.17.so
    /usr/local/lib/python3.8/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.8/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-1.21.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 61.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you h

In [2]:
!pip install -U gensim

In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import os
import gensim

In [4]:
import nltk
nltk.download('punkt')
# import nltk
nltk.download('wordnet')
# import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ckred\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ckred\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ckred\AppData\Roaming\nltk_data...


True

In [6]:
docs = []

df=pd.read_csv('train_data.csv')

for i in range(len(df['Paragraph'])):
    docs.append(df['Paragraph'][i])

    #for i in range(df.shape[0]):
    #rw=df.iloc[i]
    #data.append('Paragraph')

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(docs)]

In [7]:
print(len(docs))

75055


In [8]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [9]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [10]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [11]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [18]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 66709
Number of documents: 75055


In [13]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [14]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -4.9259.
[([(0.055977095, 'the'),
   (0.055244114, 'of'),
   (0.04389015, 'to'),
   (0.031207014, 'and'),
   (0.023090677, 'that'),
   (0.019567246, 'in'),
   (0.018908763, 'is'),
   (0.018788397, 'a'),
   (0.014685984, 'or'),
   (0.011552846, 'for'),
   (0.011112772, 'be'),
   (0.010565061, 'are'),
   (0.009365342, 'it'),
   (0.008209435, 'by'),
   (0.0081128245, 'not'),
   (0.007768786, 'this'),
   (0.0073314523, 'an'),
   (0.007137246, 'have'),
   (0.0068403846, 'with'),
   (0.005657693, 'their')],
  -0.9543623576904267),
 ([(0.16503844, 'the'),
   (0.063352145, 'of'),
   (0.04762937, 'in'),
   (0.040807404, 'and'),
   (0.023748862, 'wa'),
   (0.023151075, 'to'),
   (0.014420027, 'by'),
   (0.01294425, 'were'),
   (0.009820007, 'with'),
   (0.009667606, 'on'),
   (0.0087735625, 'from'),
   (0.008305214, 'a'),
   (0.0068181865, 'it'),
   (0.0062941182, 'at'),
   (0.006040042, 'had'),
   (0.0050693112, 'century'),
   (0.004603503, 'first'),
   (0.0045343293, '

In [15]:
#FULL

#max_epochs = 1
#vec_size = 728
#alpha = 0.025


model_full = gensim.models.ldamodel.LdaModel(corpus, num_topics=50, alpha='auto', eval_every=5)

model_full.save("ldam.model")
print("Model Saved")

Model Saved


In [16]:
import gensim.downloader as api

In [21]:
gensim.__path__[0]

'c:\\Python311\\Lib\\site-packages\\gensim'

In [ ]:
gensim.models.ensemblelda.EnsembleLda()

In [19]:
#Pretrained
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lda_train_file = os.path.join(test_data_dir, 'ensemblelda')
lda_test_file = os.path.join(test_data_dir, 'ensemblelda')
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.ldamodel.LdaModel.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lda_train_file))
test_corpus = list(read_corpus(lda_test_file, tokens_only=True))
model_pre = gensim.models.ldamodel.LdaModel(corpus, num_topics=50, alpha='auto', eval_every=5)
model_pre.train(train_corpus, total_examples=model_pre.corpus_count, epochs=model_pre.epochs)

AttributeError: type object 'LdaModel' has no attribute 'TaggedDocument'